In [1]:
import pandas as pd
import numpy as np


In [34]:
import mysql.connector

def load_stockdata(symbol):
    serverName = "localhost"
    cnx = mysql.connector.connect(user='michael', password='Password2017',
                                  host=serverName, database='stock_market')
    cursor = cnx.cursor()
    
    query = \
    "SELECT stock.*, bb.`sma20` AS bb_middle, bb.`band_upper` AS bb_upper, bb.`band_lower` AS bb_lower, \
    kc.`ema20` AS kc_middle, kc.`atr2_upper` AS kc_2_upper, kc.`atr2_lower` AS kc_2_lower, \
    kc.`atr3_upper` AS kc_3_upper, kc.`atr3_lower` AS kc_3_lower, \
    kc.`atr1_upper` AS kc_1_upper, kc.`atr1_lower` AS kc_1_lower, \
    macd.`macd_histogram` as macd, ttma.macd_histogram as wave_a, ttmb.macd_histogram as wave_b, ttmc.macd_histogram as wave_c \
    FROM \
    ( SELECT * FROM (SELECT symbol, effective_date, `adj_open`, `adj_high`, `adj_low`, `adj_close`, `adj_volume` FROM `market.stock_price` \
      WHERE symbol = '{0}' ORDER BY effective_date DESC LIMIT 260) sub ORDER BY `effective_date` ASC ) stock \
    LEFT JOIN \
    `idc.bollinger_bands` AS bb ON bb.symbol = stock.symbol AND bb.effective_date = stock.effective_date \
    LEFT JOIN \
    `idc.keltner_channels` AS kc ON kc.symbol = stock.symbol AND kc.effective_date = stock.effective_date \
    LEFT JOIN \
    `idc.macd` as macd on macd.symbol = stock.symbol and macd.effective_date = stock.effective_date \
        and macd.short_period = 12 and macd.long_period=26 and macd.signal_period=9 \
    LEFT JOIN \
    `idc.macd` as ttma on ttma.symbol = stock.symbol and ttma.effective_date = stock.effective_date \
        and ttma.short_period = 8 and ttma.long_period=34 and ttma.signal_period=34 \
    LEFT JOIN \
    `idc.macd` as ttmb on ttmb.symbol = stock.symbol and ttmb.effective_date = stock.effective_date \
        and ttmb.short_period = 8 and ttmb.long_period=89 and ttmb.signal_period=89 \
    LEFT JOIN \
    `idc.macd` as ttmc on ttmc.symbol = stock.symbol and ttmc.effective_date = stock.effective_date \
        and ttmc.short_period = 8 and ttmc.long_period=144 and ttmc.signal_period=144 \
    ".format(symbol)

    cursor.execute(query)
    dfsql = pd.DataFrame(cursor.fetchall(), dtype=np.float)
    dfsql.columns = cursor.column_names
    dfsql.set_index('effective_date', inplace=True)
    cursor.close()
    return dfsql


In [150]:
def drawInterChart(df):
    trace = go.Ohlc(x=df.date, open=df.open, high=df.high, low=df.low, close=df.close)
    data = [trace]
    layout = {
        'title': 'Stock Chart',
        'yaxis': {'title': 'Price'},
        'shapes': [{
            'x0': '2016-12-09', 'x1': '2016-12-09',
            'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper',
            'line': {'color': 'rgb(30,30,30)', 'width': 1}
        }],
        'annotations': [{
            'x': '2016-12-09', 'y': 0.05, 'xref': 'x', 'yref': 'paper',
            'showarrow': False, 'xanchor': 'left',
            'text': 'Increase Period Begins'
        }]
    }
    fig = dict(data=data, layout=layout)
    
    # 
    # embbeded chart
    #plotly.offline.iplot(fig)
    
    # standalone chart
    plotly.offline.plot(fig)
    
    # online chart 
    # py.iplot(fig, filename='appl-recession-candlestick')

In [35]:
df = load_stockdata('AAPL')
df['macd_up'] = df.macd > df.macd.shift(-1)
df.head(20)


,symbol,adj_open,adj_high,adj_low,adj_close,adj_volume,bb_middle,bb_upper,bb_lower,kc_middle,...,kc_2_lower,kc_3_upper,kc_3_lower,kc_1_upper,kc_1_lower,macd,wave_a,wave_b,wave_c,macd_up
effective_date,,,,,,,,,,,,,,,,,,,,,
2018-02-15,AAPL,169.790,173.0900,169.0000,172.99,50609595.0,166.9305,180.703680,153.157320,166.406840,...,157.404870,179.909795,152.903885,170.907825,161.905855,0.782776,-1.208818,-7.189518,-9.188372,True
2018-02-14,AAPL,163.045,167.5400,162.8800,167.37,39669178.0,167.2440,181.816823,152.671177,165.713876,...,156.899446,178.935521,152.492231,170.121091,161.306661,0.021957,-3.192245,-9.432043,-11.392292,True
2018-02-13,AAPL,161.950,164.7500,161.6500,164.34,32104756.0,167.8305,183.293392,152.367608,165.539547,...,156.764007,178.702857,152.376237,169.927317,161.151777,-0.577555,-4.644653,-10.876729,-12.730223,True
2018-02-12,AAPL,158.500,163.8900,157.5100,162.71,60560145.0,168.4230,184.210220,152.635780,165.665815,...,156.692157,179.126302,152.205328,170.152644,161.178986,-1.133196,-5.858278,-11.911275,-13.616759,True
2018-02-09,AAPL,157.070,157.8900,150.2400,155.97,66723743.0,169.1420,185.131502,153.152498,165.976953,...,157.531475,178.645170,153.308736,170.199692,161.754214,-1.717021,-6.998818,-12.749916,-14.280405,True
2018-02-08,AAPL,160.290,161.0000,155.0300,155.32,49594129.0,170.1075,185.099835,155.115165,167.030317,...,159.112111,178.907626,155.153008,170.989420,163.071214,-1.780807,-6.844619,-11.971549,-13.256167,False
2018-02-07,AAPL,163.085,163.4000,159.0685,159.54,50852130.0,171.0560,184.507250,157.604750,168.262982,...,160.654144,179.676239,156.849725,172.067401,164.458563,-1.635879,-6.202186,-10.632697,-11.664730,True
2018-02-06,AAPL,154.830,163.7200,154.0000,163.03,66625484.0,171.7955,184.220027,159.370973,169.181190,...,161.653442,180.472812,157.889568,172.945064,165.417316,-1.672409,-5.982369,-9.840878,-10.660671,True
2018-02-05,AAPL,159.100,163.8800,156.0000,157.49,66090446.0,172.3615,184.152417,160.570583,169.828684,...,163.217262,179.745817,159.911551,173.134395,166.522973,-1.927012,-6.224609,-9.605126,-10.243655,False


In [10]:
#[macd, up in df.macd_histogram, df.macd_up]


In [5]:
#[ row.macd > 0 & row.macd_up for row in df]
#df['color'] = ('green' if (df['macd'] > 0).bool() else 'red')
#[row['macd'] for row in df.iterrows()]
colordict = []
for index, row in df.iterrows():
    
    color =  '#029107' if (row['macd']>=0 and row['macd_up']==True) else \
            ('#0599b7' if (row['macd']>=0 and row['macd_up']==False) else \
            ('#871001' if (row['macd']<0 and row['macd_up']==False) else '#d35004' ))
    colordict.append(color)
    #print(index, row['macd'], row['macd_up'], color)
    

In [14]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [32]:
def multi_chart(df):
    
    style_bollingerbands_middle = dict( color = ('rgb(22, 96, 167)'), width = 1, dash = 'dot')
    style_bollingerbands_upper = dict( color = ('rgb(22, 96, 167)'), width = 1, dash = 'line')
    style_bollingerbands_lower = dict( color = ('rgb(91, 154, 255)'), width = 1, dash = 'line')

    style_kc_middle = dict( color = ('rgb(247, 94, 39)'), width = 1, dash = 'dot')
    style_kc2_upper = dict( color = ('rgb(247, 94, 39)'), width = 1, dash = 'line')
    style_kc2_lower = dict( color = ('rgb(244, 118, 73)'), width = 1, dash = 'line')

    style_kc3_upper = dict( color = ('rgb(46, 155, 53)'), width = 1, dash = 'line')
    style_kc3_lower = dict( color = ('rgb(68, 206, 77)'), width = 1, dash = 'line')

    style_kc1_upper = dict( color = ('rgb(244, 78, 66)'), width = 1, dash = 'line')
    style_kc1_lower = dict( color = ('rgb(198, 70, 61)'), width = 1, dash = 'line')

    trace_ohlc = go.Ohlc(x=df.index, open=df.adj_open, high=df.adj_high, low=df.adj_low, close=df.adj_close, 
                         showlegend=False, name='close price',
                         increasing=dict(line=dict(color= '#b1e2b6')),
                         decreasing=dict(line=dict(color= '#a89a95'))
                        )
    
    trace_bb_middle = go.Scatter(x=df.index, y=df.bb_middle, name='BB Middle', legendgroup='Bollinger Bands',
                                 showlegend=False, opacity=0.5,
                                 line = style_bollingerbands_middle
                                )
    trace_bb_upper = go.Scatter(x=df.index, y=df.bb_upper, name='BB Upper', legendgroup='Bollinger Bands',
                                line = style_bollingerbands_upper
                               )
    trace_bb_lower = go.Scatter(x=df.index, y=df.bb_lower, name='BB Lower', legendgroup='Bollinger Bands',
                                line = style_bollingerbands_lower
                               )
    
    trace_kc_middle = go.Scatter(x=df.index, y=df.kc_middle, name='KC Middle', legendgroup='Keltner Channels 2ATR',
                                 showlegend=False, opacity=0.5,
                                 line = style_kc_middle
                                )
    trace_kc_2atr_upper = go.Scatter(x=df.index, y=df.kc_2_upper, name='KC 2ATR Upper', legendgroup='Keltner Channels 2ATR',
                                     line = style_kc2_upper
                                    )
    trace_kc_2atr_lower = go.Scatter(x=df.index, y=df.kc_2_lower, name='KC 2ATR Lower', legendgroup='Keltner Channels 2ATR',
                                     line = style_kc2_lower
                                    )

    # create volume bar chart on the bottom
    tarce_volume = go.Bar(x=df.index, y=df.adj_volume, name='volume', showlegend=False, marker=dict(color='rgb(87, 91, 130)'),
                          yaxis = 'y2'
                         )

    # color dict for MACD
    colordict = []
    for index, row in df.iterrows():

        color =  '#029107' if (row['macd']>=0 and row['macd_up']==True) else \
                ('#0599b7' if (row['macd']>=0 and row['macd_up']==False) else \
                ('#871001' if (row['macd']<0 and row['macd_up']==False) else '#d35004' ))
        colordict.append(color)

    trace_macd = go.Bar(x=df.index, y=df.macd, name='MACD(12/26/9)', showlegend=False, 
                        marker=dict(color=colordict),
                        yaxis = 'y3'
                       )

    trace_wave_a = go.Bar(x=df.index, y=df.wave_a, name='WAVE A(8/34/34)', showlegend=False, 
                    marker=dict(color=['#029107' if val>=0 else '#871001' for val in df.wave_a]),
                    yaxis = 'y4'
                   )

    trace_wave_b = go.Bar(x=df.index, y=df.wave_b, name='WAVE B(8/89/89)', showlegend=False, 
                marker=dict(color=['#029107' if val>=0 else '#871001' for val in df.wave_b]),
                yaxis = 'y5'
               )
    trace_wave_c = go.Bar(x=df.index, y=df.wave_c, name='WAVE C(8/144/144)', showlegend=False, 
                    marker=dict(color=['#029107' if val>=0 else '#871001' for val in df.wave_c]),
                    yaxis = 'y6'
                   )

    
    data = [trace_ohlc, trace_bb_middle, trace_bb_upper, trace_bb_lower, 
            trace_kc_middle, trace_kc_2atr_upper, trace_kc_2atr_lower, 
            tarce_volume, trace_macd, trace_wave_a, trace_wave_b, trace_wave_c]
    
    
    axis_template = dict(showgrid=False, zeroline=True, showline=True, rangeslider=dict(visible=False))
    layout = go.Layout(
        title = 'John Carter TTM Squeeze',
        xaxis = axis_template,
        yaxis = dict(domain=[0.5,1]),
        yaxis2 = dict(domain=[0.4,0.5], visible=False),
        yaxis3 = dict(domain=[0.3,0.4]),
        yaxis4 = dict(domain=[0.2,0.3], visible=False),
        yaxis5 = dict(domain=[0.1,0.2], visible=False),
        yaxis6 = dict(domain=[0,0.1], visible=False),
        legend = dict(x=0.05, y=1.0)
    )

    fig = go.Figure(data=data, layout=layout)
    # standalone chart
    plotly.offline.plot(fig, "multi chart")


In [36]:
multi_chart(df)